In [259]:
import pandas as pd
import numpy as np

In [260]:
df_train = pd.read_csv('../analysis/train.csv')
df_label_train = pd.read_csv('../analysis/train_label.csv')
df_sensor_log = pd.read_csv('../data/sensor_log.csv')

In [261]:
df_val = pd.read_csv('../analysis/val.csv')
df_label_val = pd.read_csv('../analysis/val_label.csv')

In [262]:
df_train = pd.concat([df_train, df_val])
df_label_train = pd.concat([df_label_train, df_label_val])

In [263]:
offset = np.max(df_sensor_log['block_id'].values) + 1
offset

19283

In [264]:
block_id_anomalous = df_label_train[df_label_train['anomalous'] == 1]['block_id'].values

In [265]:
df_anomalous = df_train[df_train['block_id'].isin(block_id_anomalous)]

In [266]:
df_desc = df_anomalous.describe()

In [267]:
df_desc.head()

,block_id,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09
count,1640.000000,1640.000000,1640.000000,1640.000000,1640.000000,1640.000000,1640.000000,1640.000000,1640.000000,1640.000000,1640.000000
mean,9035.018293,2.449637,48.097757,52.060511,44.305572,631.324514,75.048025,13.827248,16.154966,15.465440,15.141328
std,5716.062547,0.190367,1.710918,1.465615,1.479873,34.549196,9.703026,0.937933,0.347999,0.369902,0.230203
min,197.000000,0.000000,40.104160,45.182290,40.190970,200.558441,0.000000,3.009259,14.712438,14.456552,14.147890
25%,4316.250000,2.449653,47.141709,51.085070,43.402775,629.513900,70.640672,13.418690,16.001160,15.183740,15.053530


In [268]:
mean = df_desc.loc['mean'].values[1:]
std = df_desc.loc['std'].values[1:]

In [269]:
data = []
for m, s in zip(mean, std):
    sample = np.random.normal(m, s, 40000).reshape(-1, 1)
    data.append(sample)
data = np.hstack(data)
data.shape

(40000, 10)

In [270]:
num_feats = df_sensor_log.columns.to_list()[2:]
df_feat = pd.DataFrame(columns=num_feats)

In [271]:
block_id = []
for i in range(data.reshape(-1,10,10).shape[0]):
    block_id.append(offset + i)

In [272]:
data_block_id = np.array(block_id)[:, None]
data_block_id = np.repeat(data_block_id, 10, 1)
data_block_id = data_block_id.reshape(-1)

data_timestamp = ['2018-04-01 00:00:00'] * 40000

In [273]:
df_block_time = pd.DataFrame(data={
    'timestamp': data_timestamp,
    'block_id': data_block_id
})
df_block_time.head()

,timestamp,block_id
0,2018-04-01 00:00:00,19283
1,2018-04-01 00:00:00,19283
2,2018-04-01 00:00:00,19283
3,2018-04-01 00:00:00,19283
4,2018-04-01 00:00:00,19283


In [274]:
df_data = pd.DataFrame(columns=num_feats, data=data)
df_data.head()

,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09
0,2.152724,48.990054,51.525976,45.775100,656.496567,67.883470,14.285256,16.120242,15.869066,15.338918
1,2.529914,46.909243,54.629688,43.239581,625.894194,71.102256,13.914366,15.987587,15.633496,15.056600
2,2.569834,47.400085,53.062656,44.724780,632.757138,68.900715,14.225144,16.475916,15.538160,15.005391
3,2.446170,46.555516,50.472203,43.017065,585.927345,78.653791,13.194951,16.399957,15.267439,15.272159
4,2.651781,50.248741,51.715692,45.265821,604.675502,76.839135,13.355820,16.227751,15.326910,15.124609


In [275]:
df_all = pd.concat([df_block_time, df_data], axis=1)
df_all.head()

,timestamp,block_id,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09
0,2018-04-01 00:00:00,19283,2.152724,48.990054,51.525976,45.775100,656.496567,67.883470,14.285256,16.120242,15.869066,15.338918
1,2018-04-01 00:00:00,19283,2.529914,46.909243,54.629688,43.239581,625.894194,71.102256,13.914366,15.987587,15.633496,15.056600
2,2018-04-01 00:00:00,19283,2.569834,47.400085,53.062656,44.724780,632.757138,68.900715,14.225144,16.475916,15.538160,15.005391
3,2018-04-01 00:00:00,19283,2.446170,46.555516,50.472203,43.017065,585.927345,78.653791,13.194951,16.399957,15.267439,15.272159
4,2018-04-01 00:00:00,19283,2.651781,50.248741,51.715692,45.265821,604.675502,76.839135,13.355820,16.227751,15.326910,15.124609


In [276]:
df_label = pd.DataFrame(columns=df_label_train.columns.to_list())

In [277]:
df_label['block_id'] = block_id
df_label['anomalous'] = [1] * 4000

In [278]:
df_all.to_csv('../analysis/new_data/new_data.csv', index=False)
df_label.to_csv('../analysis/new_data/df_new_data_label.csv', index=False)